In [2]:
import os
import pandas as pd
import pendulum
import yfinance as yf

In [2]:
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest 
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetStatus
from alpaca.trading.enums import AssetClass
from alpaca.data.enums import Adjustment
from alpaca.data.timeframe import TimeFrameUnit

In [3]:
# Environment Variables
api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_SECRET_KEY")

# Clients
data_client = StockHistoricalDataClient(api_key,secret_key)
trading_client = TradingClient(api_key,secret_key)

In [ ]:
asset_request = GetAssetsRequest(
    asset_class = AssetClass.US_EQUITY,
    status=AssetStatus.ACTIVE,
)

all_assets = trading_client.get_all_assets(asset_request)

all_assets = [vars(asset) for asset in all_assets]

df = pd.DataFrame(data=all_assets)

df = df[df['tradable'] & df['shortable'] & df['fractionable']]

data_path = "assets.csv"

df.to_csv(data_path,index=False)

In [ ]:
# Parameters
end = pendulum.datetime(2023,12,31)
start = pendulum.datetime(2020,12,31)

symbols = pd.read_csv('assets.csv')
symbols = symbols['symbol'].to_list()

bars_request = StockBarsRequest(
    symbol_or_symbols=symbols,
    timeframe=TimeFrame(1,TimeFrameUnit.Day),
    start=start,
    end=end,
    adjustment=Adjustment.ALL,
)

bars = data_client.get_stock_bars(bars_request)

df = bars.df

df = df.reset_index()


data_path = "data.csv"

df.to_csv(data_path,index=False)

df

In [4]:
bmk = yf.download('SPY', start='1994-01-01')

bmk.to_csv('bmk.csv')

[*********************100%%**********************]  1 of 1 completed


In [4]:
from db import DB

db = DB()

df = db.get_historical_data()

df.to_csv("data.csv")

Connection to the PostgreSQL database was successful.


In [4]:
df = pd.read_feather("~/Data/crsp_daily.ftr")

df = df[df['caldt'] >= '2005-01-01']

# df = df.rename(columns={'permno':'symbol', 'prc': 'close'})

# df = df.drop(columns=['shrcd','excd','siccd','vol','shr'])

# df = df.sort_values(by=['symbol','caldt'])

# df['mdt'] = pd.to_datetime(df['caldt']).dt.strftime("%Y-%m")


df

,permno,caldt,shrcd,excd,siccd,prc,ret,vol,shr
5152,10001,2005-01-03,11,3,4920,6.67000,-0.033333,17651.0,2599.0
5153,10001,2005-01-04,11,3,4920,6.51000,-0.023988,2781.0,2599.0
5154,10001,2005-01-05,11,3,4920,6.70000,0.029186,2542.0,2599.0
5155,10001,2005-01-06,11,3,4920,6.51000,-0.028358,716.0,2599.0
5156,10001,2005-01-07,11,3,4920,6.54600,0.005530,946.0,2599.0
...,...,...,...,...,...,...,...,...,...
105258375,93436,2023-12-22,11,3,9999,252.53999,-0.007701,93148541.0,3178921.0
105258376,93436,2023-12-26,11,3,9999,256.60999,0.016116,86700724.0,3178921.0
105258377,93436,2023-12-27,11,3,9999,261.44000,0.018822,106250779.0,3178921.0
105258378,93436,2023-12-28,11,3,9999,253.17999,-0.031594,113250680.0,3178921.0


In [5]:
df.to_csv("crsp_daily.csv")